# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> #### Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Módulo

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [2]:
#include <stdio.h>

int main()
{
    double parcela_inicial = 200;
    int parcelas = 5;
    double taxa = 0.01;
    double parcela_atual = parcela_inicial;
    for (int i = 1; i <= parcelas; i++)
    {
        printf("Parcela %d: %.2f\n", i, parcela_atual);
        parcela_atual = parcela_atual * (1 + taxa);
    }
    return 0;
    
}

Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [5]:
#include <stdio.h>

double proximaParcela(double valor_atual, int n_atual, int parcelas, double taxa)
{
    if (n_atual < parcelas)
    {
        if (n_atual == 0)
        {
            return valor_atual;
        }
        else
        {
            return valor_atual * (1 + taxa);
        }
    }
    else
    {
        return 0;
    }
}

int main()
{
    double parcela_inicial = 200;
    int parcelas = 5;
    double taxa = 0.01;
    double valor_atual;
    int n_atual = 0;
    valor_atual = proximaParcela(parcela_inicial, n_atual++,parcelas, taxa);
    while (valor_atual != 0)
    {
        printf("Parcela %d: %.2f\n", n_atual, valor_atual);
        valor_atual = proximaParcela(valor_atual, n_atual++, parcelas, taxa);
    }
    return 0;
}

Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12


## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [1]:
#include <stdio.h>

double PARCELA_ATUAL;
int N_PARCELA_ATUAL;
double TOTAL_PARCELAS;
double TAXA;

double parcelaInicial(double parc_inicial, int total_parcelas, double taxa)
{
    PARCELA_ATUAL = parc_inicial;
    N_PARCELA_ATUAL = 1;
    TOTAL_PARCELAS = total_parcelas;
    TAXA = taxa;
    return parc_inicial;
}

double proximaParcela(void)
{
    if (N_PARCELA_ATUAL <= TOTAL_PARCELAS)
    {
        if (N_PARCELA_ATUAL == 1)
        {
            return PARCELA_ATUAL;
        }
        else
        {
            return PARCELA_ATUAL * (1 + TAXA);
        }
    }
    else
    {
        
        return 0;
    }
}

int main()
{

    double parcela_atual;
    parcela_atual = parcelaInicial(200, 5, 0.01);
    while (parcela_atual != 0)
    {
        printf("Parcela %d: %.2f\n", N_PARCELA_ATUAL, parcela_atual);
        PARCELA_ATUAL = parcela_atual;
        N_PARCELA_ATUAL += 1;
        parcela_atual = proximaParcela();
    }
    return 0;
}

Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12


## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [1]:
#include <stdio.h>

typedef struct _parcela
{
    double valor;
    int n_parcela;
    int total_parcelas;
    double taxa;
} parcela;

double proximaParcela(parcela atual)
{
    if (atual.n_parcela + 1 <= atual.total_parcelas)
    {
        return atual.valor * (1 + atual.taxa);
    }
    else
    {
        return 0;
    }
}

parcela parcelaInicial(double parcela_inicial, int total_parcelas, double taxa)
{
    parcela p;
    p.valor = parcela_inicial;
    p.n_parcela = 1;
    p.total_parcelas = total_parcelas;
    p.taxa = taxa;
    return p;
}

int main(void)
{
    parcela parcela_atual = parcelaInicial(200, 5, 0.01);
    while (parcela_atual.valor != 0)
    {
        printf("Parcela %d: %.2f\n", parcela_atual.n_parcela, parcela_atual.valor);
        parcela_atual.valor = proximaParcela(parcela_atual);
        parcela_atual.n_parcela += 1;
    }
    return 0;
}

Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [3]:
#include <stdio.h>

typedef struct _parcela
{
    double valor;
    int n_parcela;
    int total_parcelas;
    double taxa;
} parcela;

double proximaParcela(parcela atual)
{
    if (atual.n_parcela + 1 <= atual.total_parcelas)
    {
        return atual.valor * (1 + atual.taxa);
    }
    else
    {
        return 0;
    }
}

parcela parcelaInicial(double parcela_inicial, int total_parcelas, double taxa)
{
    parcela p;
    p.valor = parcela_inicial;
    p.n_parcela = 1;
    p.total_parcelas = total_parcelas;
    p.taxa = taxa;
    return p;
}

int main(void)
{
    parcela p1 = parcelaInicial(200, 5, 0.01);
    parcela p2 = parcelaInicial(500, 7, 0.02);

    while (p1.valor > 0 || p2.valor > 0)
    {
        if (p1.valor > 0)
        {    
            printf("Empréstimo 1: Parcela %d: %.2f\n", p1.n_parcela, p1.valor);
            p1.valor = proximaParcela(p1);
            p1.n_parcela += 1;
        }
        if (p2.valor > 0)
        {    
            printf("Empréstimo 2: Parcela %d: %.2f\n", p2.n_parcela, p2.valor);
            p2.valor = proximaParcela(p2);
            p2.n_parcela += 1;
        }
    }
    return 0;
}

Empréstimo 1: Parcela 1: 200.00
Empréstimo 2: Parcela 1: 500.00
Empréstimo 1: Parcela 2: 202.00
Empréstimo 2: Parcela 2: 510.00
Empréstimo 1: Parcela 3: 204.02
Empréstimo 2: Parcela 3: 520.20
Empréstimo 1: Parcela 4: 206.06
Empréstimo 2: Parcela 4: 530.60
Empréstimo 1: Parcela 5: 208.12
Empréstimo 2: Parcela 5: 541.22
Empréstimo 2: Parcela 6: 552.04
Empréstimo 2: Parcela 7: 563.08
